# Importation des données :

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import requests
import numpy as np 
from scipy import stats

In [3]:
OECD = pd.read_excel('données/OECD - short term economic indicators.xlsx')

In [4]:
IMF = pd.read_excel('données/IMF - monetary policy.xlsx')

In [5]:
agregated_data=pd.ExcelFile('données/FRBNY-SCE-Data.xlsx')
agregated_anticipation= df1=pd.read_excel(agregated_data, 'Inflation expectations')

In [6]:
agrant = agregated_anticipation[['Unnamed: 1','Unnamed: 2']].drop([0,1,2]).reset_index().rename(columns={'index':'date','Unnamed: 1':'short exp','Unnamed: 2':'mid exp'})

In [7]:
Enq13 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-13-16.xlsx')

In [8]:
Enq17 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-17-19.xlsx')

In [9]:
Latest=pd.ExcelFile('https://www.newyorkfed.org/medialibrary/Interactives/sce/sce/downloads/data/frbny-sce-public-microdata-latest.xlsx')
Enq20=pd.read_excel(Latest, 'Data')

In [10]:
Enq20

,"Source: Survey of Consumer Expectations, © 2013-23 Federal Reserve Bank of New York (FRBNY). The SCE data are available without charge at www.newyorkfed.org and may be used subject to license terms posted there. FRBNY disclaims any responsibility or legal liability for this analysis and interpretation of Survey of Consumer Expectations data.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219
0,date,userid,tenure,weight,Q1,Q2,Q3,Q4new,Q5new,Q6new,...,Q48,QRA1,QRA2,survey_date,_AGE_CAT,_NUM_CAT,_REGION_CAT,_COMMUTING_ZONE,_EDU_CAT,_HH_INC_CAT
1,202001,70103231,12,0.81825,4,3,20,62,26,44,...,5,NaN,NaN,2020-01-23 00:00:00,40 to 60,High,West,218,College,Over 100k
2,202001,70103252,11,2.879998,3,2,1,30,1,10,...,2,NaN,NaN,2020-01-02 00:00:00,40 to 60,Low,Northeast,203,High School,Under 50k
3,202001,70103258,12,0.549674,4,4,0,3,4,18,...,4,NaN,NaN,2020-01-07 00:00:00,Over 60,High,South,102,College,50k to 100k
4,202001,70103285,12,0.481746,4,4,10,10,10,10,...,1,NaN,NaN,2020-01-27 00:00:00,Under 40,High,Midwest,100,College,50k to 100k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35136,202204,75005976,1,0.954878,4,5,0,5,25,55,...,5,4,7,2022-04-12 00:00:00,40 to 60,Low,South,54,Some College,50k to 100k
35137,202204,75005980,1,0.708391,2,3,34,22,86,54,...,4,3,3,2022-04-04 00:00:00,Over 60,High,West,902,College,50k to 100k
35138,202204,75005987,1,0.915896,2,4,81,70,51,47,...,5,5,4,2022-04-10 00:00:00,Under 40,Low,South,84,Some College,Under 50k
35139,202204,75005995,1,0.888988,4,4,95,5,5,5,...,3,5,4,2022-04-20 00:00:00,Under 40,Low,South,102,Some College,50k to 100k


In [11]:
#Il faut faire 2-3 manips pour rendre les bases utilisables
new_columns = Enq13.iloc[0]
Enq13.columns = new_columns
Enq13 = Enq13.drop(0)

new_columns = Enq17.iloc[0]
Enq17.columns = new_columns
Enq17 = Enq17.drop(0)

In [12]:
new_columns = Enq20.iloc[0]
Enq20.columns = new_columns
Enq20 = Enq20.drop(0)

In [13]:
Enq17['date'] = pd.to_datetime(Enq17['date'].astype(str) + '01', format='%Y%m%d')
Enq13['date'] = pd.to_datetime(Enq13['date'].astype(str) + '01', format='%Y%m%d')
Enq20['date'] = pd.to_datetime(Enq20['date'].astype(str) + '01', format='%Y%m%d')

In [14]:
Base_finale=pd.concat([Enq13,Enq17,Enq20])

In [ ]:
#Base_finale.to_excel('Base_finale.xlsx')

# I-Première spécification :

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

In [17]:
AA=Base_finale

In [18]:
list(AA.columns)

['date',
 'userid',
 'tenure',
 'weight',
 'Q1',
 'Q2',
 'Q3',
 'Q4new',
 'Q5new',
 'Q6new',
 'Q8v2',
 'Q8v2part2',
 'Q9_cent25',
 'Q9_cent50',
 'Q9_cent75',
 'Q9_var',
 'Q9_mean',
 'Q9_iqr',
 'Q9_probdeflation',
 'Q9_bin1',
 'Q9_bin2',
 'Q9_bin3',
 'Q9_bin4',
 'Q9_bin5',
 'Q9_bin6',
 'Q9_bin7',
 'Q9_bin8',
 'Q9_bin9',
 'Q9_bin10',
 'Q9bv2',
 'Q9bv2part2',
 'Q9c_cent25',
 'Q9c_cent50',
 'Q9c_cent75',
 'Q9c_var',
 'Q9c_mean',
 'Q9c_iqr',
 'Q9c_probdeflation',
 'Q9c_bin1',
 'Q9c_bin2',
 'Q9c_bin3',
 'Q9c_bin4',
 'Q9c_bin5',
 'Q9c_bin6',
 'Q9c_bin7',
 'Q9c_bin8',
 'Q9c_bin9',
 'Q9c_bin10',
 'Q10_1',
 'Q10_2',
 'Q10_3',
 'Q10_4',
 'Q10_5',
 'Q10_6',
 'Q10_7',
 'Q10_8',
 'Q10_9',
 'Q10_10',
 'Q11',
 'Q12new',
 'ES1_1',
 'ES1_2',
 'ES1_3',
 'ES1_4',
 'ES2',
 'ES3new',
 'ES4',
 'ES5',
 'Q13new',
 'Q14new',
 'Q15',
 'Q16',
 'Q17new',
 'Q18new',
 'Q19',
 'Q20new',
 'Q21new',
 'Q22new',
 'Q23v2',
 'Q23v2part2',
 'Q24_cent25',
 'Q24_cent50',
 'Q24_cent75',
 'Q24_var',
 'Q24_mean',
 'Q24_iqr',
 'Q

In [19]:
educ=pd.get_dummies(AA['_EDU_CAT'])

In [20]:
age=pd.get_dummies(AA['_AGE_CAT'])

In [21]:
income=pd.get_dummies(AA['_HH_INC_CAT'])

In [22]:
numeracy=pd.get_dummies(AA['_NUM_CAT'])

In [23]:
np.sum(educ['High School'])

15979

In [24]:
BC = pd.concat([AA, educ], axis=1).reindex(AA.index) #BC est pour Base complète. Ici, on complète la base avec des dummies

In [25]:
BC = pd.concat([BC, age], axis=1).reindex(BC.index)

In [26]:
BC = pd.concat([BC, income], axis=1).reindex(BC.index)

In [27]:
BC = pd.concat([BC, numeracy], axis=1).reindex(BC.index)

On code maintenant le genre :

In [40]:
%%capture
gender=pd.get_dummies(AA['Q33'])

In [29]:
gender.rename(columns={1: "fem"}, inplace=True)

In [30]:
BC = pd.concat([BC, gender['fem']], axis=1).reindex(BC.index)

C'est fait pour le genre. 

Faisons le ménage dans les non-réponses et dans les outliers :

In [31]:
BC=BC.dropna(subset=['Q8v2part2','Q9bv2part2']) #On supprime les personnes qui n'ont pas répondu aux questions sur les anticipations d'inflation. 

Supprimons les outliers

In [32]:
test=BC[BC['Q8v2part2']<=20]
test=test[test['Q8v2part2']>=-20]

In [33]:
max(test['Q8v2part2'])

20

In [34]:
model = OLS(BC['Q8v2part2'].astype('float'),BC['fem'].astype('float'))
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Q8v2part2   R-squared (uncentered):                   0.003
Model:                            OLS   Adj. R-squared (uncentered):              0.003
Method:                 Least Squares   F-statistic:                              399.6
Date:                Sat, 11 Mar 2023   Prob (F-statistic):                    8.78e-89
Time:                        09:25:59   Log-Likelihood:                     -7.4034e+05
No. Observations:              138457   AIC:                                  1.481e+06
Df Residuals:                  138456   BIC:                                  1.481e+06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
fem           10.5295      0.527     19.991      0.000       9.497      11.562
==============================================================================
Omnibus:                   667733.766   Durbin-Watson:                   1.391
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    8970835143631.576
Skew:                         190.590   Prob(JB):                         0.00
Kurtosis:                   39434.592   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Calcul de l'inflation :

In [35]:
inflation_US = OECD[(OECD['Country'] == "United States") & (OECD['Subject2'] == "Consumer prices: all items")]
inflation_US2 = pd.DataFrame({
    'day': pd.to_datetime(inflation_US['Time'], format='%Y-%m-%d'),
    'Inflation': inflation_US['Value'].astype(float)
})

inflation_US3 = pd.DataFrame({
    'date': inflation_US2['day'],
    'tx_evol_ann_pct': (inflation_US2['Inflation'] / inflation_US2['Inflation'].shift(12) - 1) * 100
})


In [36]:
test['inflation t']=0

In [37]:
df3 = pd.merge(test,inflation_US3,how='outer',left_on=['date'],right_on=['date'])


In [38]:
df3=df3[df3['Q1'].notnull()]

In [39]:
BF=df3

## Définition d'un indicateur d'instabilité personelle :

On va définir les variables qu'il serait pertinent d'inclure : 
- Q3 : proba changement de résidence dans les douze prochain mois
- Q10 : 3 : chomeurs
- Q13new: proba de changer d'emploi
- Q25v2 : 3 : anticipe baisse de revenus
- Q30new : proba de ne pas réussir à payer un prêt



In [41]:
BF.loc[BF['Q25v2']==3, 'loss expectation'] = 1 #On binarise le fait d'anticiper une baisse de ses revenus

In [47]:
BF['indic_perso']=(BF.fillna(0)['Q3']/100+BF.fillna(0)['Q10_3']+BF.fillna(0)['Q13new']/100+BF.fillna(0)['loss expectation']+BF.fillna(0)['Q30new']/100)/5

*j'ai remplacé le 4 de la division par 5 pcq je crois qu'il y a 5 éléments, mais si c'est pas bien supprime ; dans tous les cas ça changera pas grand-chose à la régression puisque c'est proportionnel*

*par contre c'est pas un peu chaud de remplacer tous les nan par des 0?*

Faisons quelques statistiques descriptives dessus :

In [48]:
import statistics as stat

In [49]:
stat.median(BF['indic_perso'])

0.05

In [50]:
stat.mean(BF['indic_perso'])

0.10456351316639555

In [51]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

In [52]:
weighted_average(BF, 'indic_perso', 'weight')

0.10627852801966672

In [53]:
stat.pstdev(BF['indic_perso'])

0.12198107331351413

In [54]:
len(BF['Q8v2part2'])

128471

In [55]:
Y = BF['Q8v2part2'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1529.
Date:                Sat, 11 Mar 2023   Prob (F-statistic):               0.00
Time:                        09:35:20   Log-Likelihood:            -3.9227e+05
No. Observations:              128471   AIC:                         7.845e+05
Df Residuals:                  128468   BIC:                         7.846e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.9990      0.026    116.671      0.000       2.949       3.049
tx_evol_ann_pct     0.4303      0.008     52.656      0.000       0.414       0.446
indic_perso         2.2181      0.117     18.903      0.000       1.988       2.448
==============================================================================
Omnibus:                    13278.796   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           110552.503
Skew:                           0.108   Prob(JB):                         0.00
Kurtosis:                       7.539   Cond. No.                         23.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Alternative : on sépare les différents aspects de l'instabilité personnelle pour ne pas leur donner un poids arbitraire

In [56]:
#Allure proba de changement de résidence sous 12 mois
BF['Q3']

0         50
1         25
2          3
3          0
4         10
          ..
128512     3
128513    34
128514    81
128515    95
128516     1
Name: Q3, Length: 128471, dtype: object

In [57]:
print("nb de valeurs manquantes dans Q3 : "+str(BF['Q3'].isnull().sum()))

nb de valeurs manquantes dans Q3 : 440


In [58]:
#Allure de l'indicatrice du chômage
BF['Q10_3']

0         0
1         0
2         0
3         0
4         0
         ..
128512    0
128513    0
128514    1
128515    0
128516    0
Name: Q10_3, Length: 128471, dtype: object

In [59]:
print("nb de valeurs manquantes dans Q10_3 : "+str(BF['Q10_3'].isnull().sum()))

nb de valeurs manquantes dans Q10_3 : 0


le chômage est tjrs renseigné, c'est cool

In [60]:
#Allure proba changer d'emploi
BF['Q13new']

0         NaN
1         NaN
2         NaN
3         NaN
4          30
         ... 
128512      0
128513    NaN
128514    NaN
128515      5
128516    NaN
Name: Q13new, Length: 128471, dtype: object

In [61]:
print("nb de valeurs manquantes dans Q13new : "+str(BF['Q13new'].isnull().sum()))

nb de valeurs manquantes dans Q13new : 52658


*c'est une catastrophe absolue, quasiment la moitié des participants n'y ont pas répondu...
il faudra sans doute faire une régression qui ne le prend pas en compte*

In [62]:
#Allure anticipation baisse revenus
BF['Q25v2']

0         3
1         1
2         1
3         1
4         3
         ..
128512    1
128513    3
128514    1
128515    1
128516    1
Name: Q25v2, Length: 128471, dtype: object

In [57]:
print("nb de valeurs manquantes dans Q25v2 : "+str(BF['Q25v2'].isnull().sum()))

nb de valeurs manquantes dans Q25v2 : 60


In [58]:
#Allure proba ne pas réussir à rembourser un prêt
BF['Q30new']

0         30
1          0
2         97
3          0
4          5
          ..
128512     0
128513     0
128514    61
128515     1
128516    11
Name: Q30new, Length: 128471, dtype: object

In [59]:
print("nb de valeurs manquantes dans Q30new : "+str(BF['Q30new'].isnull().sum()))

nb de valeurs manquantes dans Q30new : 226


In [63]:
BF

,date,userid,tenure,weight,Q1,Q2,Q3,Q4new,Q5new,Q6new,...,50k to 100k,Over 100k,Under 50k,High,Low,fem,inflation t,tx_evol_ann_pct,loss expectation,indic_perso
0,2013-06-01,70000220,6,16.327999,3,3,50,20,10,15,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.754417,1.0,0.360
1,2013-06-01,70000224,7,0.228,4,4,25,10,25,75,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.754417,NaN,0.050
2,2013-06-01,70000234,6,4.066,4,3,3,9,20,20,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.754417,NaN,0.200
3,2013-06-01,70000238,6,3.035,3,3,0,10,5,70,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.754417,NaN,0.000
4,2013-06-01,70000239,7,1.947,2,3,10,20,20,50,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.754417,1.0,0.290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128512,2022-04-01,75005974,1,0.556409,4,3,3,72,90,51,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,8.258629,NaN,0.006
128513,2022-04-01,75005980,1,0.708391,2,3,34,22,86,54,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,8.258629,1.0,0.268
128514,2022-04-01,75005987,1,0.915896,2,4,81,70,51,47,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,8.258629,NaN,0.484
128515,2022-04-01,75005995,1,0.888988,4,4,95,5,5,5,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,8.258629,NaN,0.202


In [64]:
instabilité = ['Q3', 'Q10_3', 'Q13new', 'Q25v2', 'Q30new']

In [65]:
#on fabrique un dataframe sans les valeurs manquantes
BF2 = BF.dropna(subset=instabilité)

In [66]:
%%capture
#du coup on re binarise
BF2.loc[BF['Q25v2']==3, 'loss expectation'] = 1
BF2['loss expectation']=BF2.fillna(0)['loss expectation']

In [67]:
instabilité = ['Q3', 'Q10_3', 'Q13new', 'loss expectation', 'Q30new']

In [69]:
%%capture
#transformation des colonnes en float
for element in instabilité :
    BF2[element] = BF2[element].astype(float)

In [71]:
BF2 = BF2.rename(columns={'Q3': 'proba chgt résidence',
                          'Q10_3': 'chômage',
                          'Q13new': 'proba chgt emploi',
                          'loss expectation': 'proba perte revenus',
                          'Q30new': 'proba ne pas réussir à rembourser un prêt'})

In [72]:
instabilité = ['proba chgt résidence',
               'chômage',
               'proba chgt emploi',
               'proba perte revenus',
               'proba ne pas réussir à rembourser un prêt']

In [75]:
Y = BF2['Q8v2part2'].astype('float')
X = BF2[instabilité]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     89.94
Date:                Sat, 11 Mar 2023   Prob (F-statistic):           1.11e-94
Time:                        09:40:02   Log-Likelihood:            -2.2895e+05
No. Observations:               75642   AIC:                         4.579e+05
Df Residuals:                   75636   BIC:                         4.580e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                         3.7076      0.026    144.931      0.000       3.657       3.758
proba chgt résidence                         -0.0031      0.001     -5.023      0.000      -0.004      -0.002
chômage                                      -1.9208      0.641     -2.999      0.003      -3.176      -0.665
proba chgt emploi                             0.0030      0.001      3.070      0.002       0.001       0.005
proba perte revenus                           0.3026      0.065      4.676      0.000       0.176       0.429
proba ne pas réussir à rembourser un prêt     0.0175      0.001     18.660      0.000       0.016       0.019
==============================================================================
Omnibus:                     8323.181   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            72011.025
Skew:                           0.165   Prob(JB):                         0.00
Kurtosis:                       7.768   Cond. No.                     1.40e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Les coeff sont tous significatifs à 95%, mais très petits

## Maintenant, essayons de définir un indicateur pour l'instabilité macro :

On va noter ici les variables qui pourraient être utiles pour définir un tel indicateur :
- Q4new: probabilité d'augmentation du chômage dans les douze prochain mois
- Q5new: proba d'une augmentation des taux d'intêret
- Q6new: proba d'une augmentation des indices boursiers: 100-Q6new = chance d'une baisse donc


In [ ]:
BF['indic_macro']=(BF.fillna(0)['Q4new']/100+BF.fillna(0)['Q5new']/100+(100-BF.fillna(0)['Q6new'])/100)/3

Maintenant, faisons quelques statistiques descriptives :

In [120]:
stat.median(BF['indic_macro'])

0.41333333333333333

In [121]:
stat.mean(BF['indic_macro'])

0.42187555946478195

In [122]:
weighted_average(BF, 'indic_macro', 'weight')

0.41943502490502443

In [123]:
stat.pstdev(BF['indic_macro'])

0.12286288554204154

In [119]:
BF['indic_macro']

0         0.383333
1         0.200000
2         0.363333
3         0.150000
4         0.300000
            ...   
128512    0.703333
128513    0.513333
128514    0.580000
128515    0.350000
128516    0.466667
Name: indic_macro, Length: 128471, dtype: float64

In [124]:
Y = BF['Q8v2part2'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     1167.
Date:                Fri, 10 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:32:20   Log-Likelihood:            -3.9205e+05
No. Observations:              128471   AIC:                         7.841e+05
Df Residuals:                  128467   BIC:                         7.841e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.0093      0.054     37.180      0.000       1.903       2.115
tx_evol_ann_pct     0.4285      0.008     52.514      0.000       0.412       0.444
indic_perso         1.4846      0.095     15.670      0.000       1.299       1.670
indic_macro         2.4449      0.117     20.808      0.000       2.215       2.675
==============================================================================
Omnibus:                    13293.140   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           110904.274
Skew:                           0.107   Prob(JB):                         0.00
Kurtosis:                       7.547   Cond. No.                         25.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Idem, alternative sans bloc :

In [76]:
BF['Q4new']

0         20
1         10
2          9
3         10
4         20
          ..
128512    72
128513    22
128514    70
128515     5
128516    50
Name: Q4new, Length: 128471, dtype: object

In [78]:
print("nb de valeurs manquantes dans Q4new : "+str(BF['Q4new'].isnull().sum()))

nb de valeurs manquantes dans Q4new : 255


In [79]:
BF['Q5new']

0         10
1         25
2         20
3          5
4         20
          ..
128512    90
128513    86
128514    51
128515     5
128516    30
Name: Q5new, Length: 128471, dtype: object

In [80]:
print("nb de valeurs manquantes dans Q5new : "+str(BF['Q5new'].isnull().sum()))

nb de valeurs manquantes dans Q5new : 241


In [81]:
BF['Q6new']

0         15
1         75
2         20
3         70
4         50
          ..
128512    51
128513    54
128514    47
128515     5
128516    40
Name: Q6new, Length: 128471, dtype: object

In [82]:
print("nb de valeurs manquantes dans Q6new : "+str(BF['Q6new'].isnull().sum()))

nb de valeurs manquantes dans Q6new : 810


Au total on va perdre pas mal de répondants, mais ça reste pas énorme

Se posera tout de même la question, si l'on envisage une grosse régression, du nombre d'individus qui auront répondu à tout ce qui nous intéresse ; d'ailleurs étudions ça avant de passer à la suite

In [111]:
len(BF[BF['Q3'].isna() | 
       BF['Q10_3'].isna()  |
       BF['Q13new'].isna() | 
       BF['Q25v2'].isna() | 
       BF['Q30new'].isna() | 
       BF['Q4new'].isna() | 
       BF['Q5new'].isna() | 
       BF['Q6new'].isna()])

53191

On perd la moitié des répondants. Tandis que sans la question de la proba de changer d'emploi :

In [112]:
len(BF[BF['Q3'].isna() |
       BF['Q10_3'].isna() |
       BF['Q25v2'].isna() |
       BF['Q30new'].isna() |
       BF['Q4new'].isna() | 
       BF['Q5new'].isna() |
       BF['Q6new'].isna()])

1179

Ce qui est beaucoup plus raisonnable. On l'exclura donc sans doute.

revenons à nos moutons

In [83]:
instabilité2 = ['Q4new', 'Q5new', 'Q6new']

In [84]:
#on fabrique un dataframe sans les valeurs manquantes
BF3 = BF.dropna(subset=instabilité2)

In [89]:
%%capture
#transformation des colonnes en float
for element in instabilité2 :
    BF3[element] = BF3[element].astype(float)

In [91]:
%%capture
#obtention d'une colonne proba diminution indices boursiers
BF3['Q6new_bis'] = 100-BF3['Q6new']

In [110]:
BF3 = BF3.rename(columns={'Q4new': 'proba augmentation chô sous douze mois',
                          'Q5new': 'proba augmentation tx intêret',
                          'Q6new_bis' : 'proba diminution indices boursiers'})

In [114]:
instabilité2 = ['proba augmentation chô sous douze mois',
                'proba augmentation tx intêret',
                'proba diminution indices boursiers']

In [97]:
Y = BF3['Q8v2part2'].astype('float')
X = BF3[instabilité2]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1142.
Date:                Sat, 11 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:06:38   Log-Likelihood:            -3.8932e+05
No. Observations:              127580   AIC:                         7.787e+05
Df Residuals:                  127576   BIC:                         7.787e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      2.4444      0.057     43.090      0.000       2.333       2.556
proba augmentation chô sous douze mois     0.0260      0.001     41.138      0.000       0.025       0.027
proba augmentation tx intêret             -0.0134      0.001    -21.843      0.000      -0.015      -0.012
proba diminution indices boursiers         0.0206      0.001     30.232      0.000       0.019       0.022
==============================================================================
Omnibus:                    12909.321   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           108578.203
Skew:                           0.027   Prob(JB):                         0.00
Kurtosis:                       7.519   Cond. No.                         308.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Tous significatifs à 95%, mais coefficients ridicules

## Tentative de maxi régression

On va inclure tout sans bloc ; sans considérer la question qui fait perdre la moitié des répondants

In [104]:
instabilité_micro = ['Q3', 'Q10_3', 'Q25v2', 'Q30new']
instabilité_macro = ['Q4new', 'Q5new', 'Q6new']
régresseurs = instabilité_micro + instabilité_macro + ['tx_evol_ann_pct', 'fem']

In [105]:
#retrait des valeurs manquantes
BF4 = BF.dropna(subset=régresseurs)

In [107]:
%%capture
#du coup on re binarise Q25v2
BF4.loc[BF['Q25v2']==3, 'loss expectation'] = 1
BF4['loss expectation']=BF4.fillna(0)['loss expectation']

In [121]:
%%capture
#obtention d'une colonne proba diminution indices boursiers
BF4['Q6new_bis'] = 100-BF4['Q6new']
BF4['Q6new_bis'] = BF4['Q6new_bis'].astype(float)

In [122]:
%%capture
#transformation des colonnes en float
for element in régresseurs :
    BF4[element] = BF4[element].astype(float)

In [123]:
BF4 = BF4.rename(columns={'Q4new': 'proba augmentation chô sous douze mois', 
                          'Q5new': 'proba augmentation tx intêret',
                          'Q6new_bis' : 'proba diminution indices boursiers',
                          'Q3': 'proba chgt résidence',
                          'Q10_3': 'chômage',
                          'loss expectation': 'proba perte revenus',
                          'Q30new': 'proba ne pas réussir à rembourser un prêt'})

In [124]:
régresseurs = ['proba augmentation chô sous douze mois',
               'proba augmentation tx intêret',
               'proba diminution indices boursiers',
               'proba chgt résidence',
               'chômage',
               'proba perte revenus',
               'proba ne pas réussir à rembourser un prêt',
               'fem',
               'tx_evol_ann_pct']

In [125]:
Y = BF4['Q8v2part2'].astype('float')
X = BF4[régresseurs]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Q8v2part2   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     826.8
Date:                Sat, 11 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:24:21   Log-Likelihood:            -3.8647e+05
No. Observations:              127292   AIC:                         7.730e+05
Df Residuals:                  127282   BIC:                         7.730e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                         1.4521      0.059     24.682      0.000       1.337       1.567
proba augmentation chô sous douze mois        0.0249      0.001     39.125      0.000       0.024       0.026
proba augmentation tx intêret                -0.0141      0.001    -23.291      0.000      -0.015      -0.013
proba diminution indices boursiers            0.0095      0.000     28.275      0.000       0.009       0.010
proba diminution indices boursiers            0.0095      0.000     28.275      0.000       0.009       0.010
proba chgt résidence                         -0.0029      0.001     -5.709      0.000      -0.004      -0.002
chômage                                       0.3440      0.079      4.366      0.000       0.190       0.498
proba perte revenus                           0.2960      0.043      6.871      0.000       0.212       0.380
proba ne pas réussir à rembourser un prêt     0.0150      0.001     20.564      0.000       0.014       0.016
fem                                           1.1239      0.061     18.372      0.000       1.004       1.244
tx_evol_ann_pct                               0.4489      0.008     55.721      0.000       0.433       0.465
==============================================================================
Omnibus:                    13847.084   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           127475.132
Skew:                          -0.078   Prob(JB):                         0.00
Kurtosis:                       7.900   Cond. No.                     1.48e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.81e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

mouais apres on peut voir avec des train/tests

# La deuxième spécification :

In [131]:
list(BF.head())

['date',
 'userid',
 'tenure',
 'weight',
 'Q1',
 'Q2',
 'Q3',
 'Q4new',
 'Q5new',
 'Q6new',
 'Q8v2',
 'Q8v2part2',
 'Q9_cent25',
 'Q9_cent50',
 'Q9_cent75',
 'Q9_var',
 'Q9_mean',
 'Q9_iqr',
 'Q9_probdeflation',
 'Q9_bin1',
 'Q9_bin2',
 'Q9_bin3',
 'Q9_bin4',
 'Q9_bin5',
 'Q9_bin6',
 'Q9_bin7',
 'Q9_bin8',
 'Q9_bin9',
 'Q9_bin10',
 'Q9bv2',
 'Q9bv2part2',
 'Q9c_cent25',
 'Q9c_cent50',
 'Q9c_cent75',
 'Q9c_var',
 'Q9c_mean',
 'Q9c_iqr',
 'Q9c_probdeflation',
 'Q9c_bin1',
 'Q9c_bin2',
 'Q9c_bin3',
 'Q9c_bin4',
 'Q9c_bin5',
 'Q9c_bin6',
 'Q9c_bin7',
 'Q9c_bin8',
 'Q9c_bin9',
 'Q9c_bin10',
 'Q10_1',
 'Q10_2',
 'Q10_3',
 'Q10_4',
 'Q10_5',
 'Q10_6',
 'Q10_7',
 'Q10_8',
 'Q10_9',
 'Q10_10',
 'Q11',
 'Q12new',
 'ES1_1',
 'ES1_2',
 'ES1_3',
 'ES1_4',
 'ES2',
 'ES3new',
 'ES4',
 'ES5',
 'Q13new',
 'Q14new',
 'Q15',
 'Q16',
 'Q17new',
 'Q18new',
 'Q19',
 'Q20new',
 'Q21new',
 'Q22new',
 'Q23v2',
 'Q23v2part2',
 'Q24_cent25',
 'Q24_cent50',
 'Q24_cent75',
 'Q24_var',
 'Q24_mean',
 'Q24_iqr',
 'Q

On doit définir les intervalles qui vont nous intéresser ici :
- pour l'inflation à court terme: Q9 inflation supérieur à 4% (bin 1, bin 2 et bin 3)
- déflation : bin6 à bin 10

- long terme: Q9c+les mêmes bin

N.B : On considère pour la suite que les anticipations non remplies sont de 10 ( équiprobabilit de chaque tranche )

In [139]:
BF['proba_st_high']=(BF['Q9_bin1'].fillna(10)+BF['Q9_bin2'].fillna(10)+BF['Q9_bin3'].fillna(10))

/tmp/ipykernel_198/2054418336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF['proba_st_high']=(BF['Q9_bin1'].fillna(10)+BF['Q9_bin2'].fillna(10)+BF['Q9_bin3'].fillna(10))


1135

In [140]:
BF['proba_st_defl']=(BF['Q9_bin6'].fillna(10)+BF['Q9_bin7'].fillna(10)+BF['Q9_bin8'].fillna(10)+BF['Q9_bin9'].fillna(10)+BF['Q9_bin10'].fillna(10))

/tmp/ipykernel_198/9693778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF['proba_st_defl']=(BF['Q9_bin6'].fillna(10)+BF['Q9_bin7'].fillna(10)+BF['Q9_bin8'].fillna(10)+BF['Q9_bin9'].fillna(10)+BF['Q9_bin10'].fillna(10))


In [141]:
BF['proba_lt_high']=(BF['Q9c_bin1'].fillna(10)+BF['Q9c_bin2'].fillna(10)+BF['Q9c_bin3'].fillna(10))

/tmp/ipykernel_198/3420240262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF['proba_lt_high']=(BF['Q9c_bin1'].fillna(10)+BF['Q9c_bin2'].fillna(10)+BF['Q9c_bin3'].fillna(10))


In [142]:
BF['proba_lt_defl']=(BF['Q9c_bin6'].fillna(10)+BF['Q9c_bin7'].fillna(10)+BF['Q9c_bin8'].fillna(10)+BF['Q9c_bin9'].fillna(10)+BF['Q9c_bin10'].fillna(10))

/tmp/ipykernel_198/3127198973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF['proba_lt_defl']=(BF['Q9c_bin6'].fillna(10)+BF['Q9c_bin7'].fillna(10)+BF['Q9c_bin8'].fillna(10)+BF['Q9c_bin9'].fillna(10)+BF['Q9c_bin10'].fillna(10))


### A - Short term - high :

In [137]:
BF.rename(columns={'High': "High_num"}, inplace=True)

/tmp/ipykernel_198/2587465590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF.rename(columns={'High': "High_num"}, inplace=True)


1110

In [154]:
Y = BF['proba_st_high'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          proba_st_high   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     1320.
Date:                Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                        09:51:49   Log-Likelihood:            -6.3084e+05
No. Observations:              128471   AIC:                         1.262e+06
Df Residuals:                  128460   BIC:                         1.262e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              20.6750      0.417     49.537      0.000      19.857      21.493
tx_evol_ann_pct     4.3132      0.052     82.284      0.000       4.210       4.416
indic_perso        12.6546      0.622     20.347      0.000      11.436      13.874
indic_macro        25.6988      0.756     33.974      0.000      24.216      27.181
fem                 8.2663      0.396     20.863      0.000       7.490       9.043
Over 60             2.0917      0.221      9.444      0.000       1.658       2.526
Under 40           -4.0078      0.230    -17.445      0.000      -4.458      -3.557
College            -5.3486      0.202    -26.417      0.000      -5.745      -4.952
Over 100k          -3.5887      0.231    -15.536      0.000      -4.041      -3.136
Under 50k           3.0681      0.227     13.523      0.000       2.623       3.513
High_num           -5.5680      0.222    -25.050      0.000      -6.004      -5.132
==============================================================================
Omnibus:                    11364.957   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11603.145
Skew:                           0.685   Prob(JB):                         0.00
Kurtosis:                       2.460   Cond. No.                         27.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### B - Short term - deflation :

In [155]:
Y = BF['proba_st_defl'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          proba_st_defl   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     480.2
Date:                Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                        09:53:31   Log-Likelihood:            -5.8582e+05
No. Observations:              128471   AIC:                         1.172e+06
Df Residuals:                  128460   BIC:                         1.172e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              20.9418      0.294     71.237      0.000      20.366      21.518
tx_evol_ann_pct    -0.7865      0.037    -21.301      0.000      -0.859      -0.714
indic_perso         4.3466      0.438      9.922      0.000       3.488       5.205
indic_macro        -4.3406      0.533     -8.147      0.000      -5.385      -3.296
fem                 4.7500      0.279     17.020      0.000       4.203       5.297
Over 60            -1.7483      0.156    -11.207      0.000      -2.054      -1.443
Under 40            2.3371      0.162     14.442      0.000       2.020       2.654
College            -2.4732      0.143    -17.342      0.000      -2.753      -2.194
Over 100k          -0.5408      0.163     -3.324      0.001      -0.860      -0.222
Under 50k           2.3332      0.160     14.600      0.000       2.020       2.646
High_num           -5.7936      0.157    -37.005      0.000      -6.101      -5.487
==============================================================================
Omnibus:                    65508.685   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           534198.947
Skew:                           2.329   Prob(JB):                         0.00
Kurtosis:                      11.837   Cond. No.                         27.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### C-Long terme - high :

In [158]:
Y = BF['proba_lt_high'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          proba_lt_high   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     698.9
Date:                Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                        09:55:47   Log-Likelihood:            -6.3374e+05
No. Observations:              128471   AIC:                         1.268e+06
Df Residuals:                  128460   BIC:                         1.268e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              24.6136      0.427     57.658      0.000      23.777      25.450
tx_evol_ann_pct     1.9839      0.054     37.003      0.000       1.879       2.089
indic_perso        10.9970      0.636     17.287      0.000       9.750      12.244
indic_macro        24.8554      0.774     32.126      0.000      23.339      26.372
fem                 7.2306      0.405     17.842      0.000       6.436       8.025
Over 60             3.0667      0.227     13.537      0.000       2.623       3.511
Under 40           -3.9076      0.235    -16.629      0.000      -4.368      -3.447
College            -4.6713      0.207    -22.557      0.000      -5.077      -4.265
Over 100k          -3.0472      0.236    -12.898      0.000      -3.510      -2.584
Under 50k           3.2382      0.232     13.954      0.000       2.783       3.693
High_num           -4.8301      0.227    -21.245      0.000      -5.276      -4.384
==============================================================================
Omnibus:                    17542.763   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12174.561
Skew:                           0.645   Prob(JB):                         0.00
Kurtosis:                       2.218   Cond. No.                         27.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### D - Long term - Deflation :

In [161]:
Y = BF['proba_lt_defl'].astype('float')
X = BF.fillna(0)[['tx_evol_ann_pct','indic_perso','indic_macro','fem','Over 60','Under 40','College','Over 100k','Under 50k','High_num']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
su = results.summary()
su

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          proba_lt_defl   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     423.0
Date:                Sun, 12 Mar 2023   Prob (F-statistic):               0.00
Time:                        09:58:00   Log-Likelihood:            -5.9836e+05
No. Observations:              128471   AIC:                         1.197e+06
Df Residuals:                  128460   BIC:                         1.197e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              20.2868      0.324     62.589      0.000      19.652      20.922
tx_evol_ann_pct     0.3964      0.041      9.738      0.000       0.317       0.476
indic_perso         5.3364      0.483     11.048      0.000       4.390       6.283
indic_macro        -2.2485      0.587     -3.828      0.000      -3.400      -1.097
fem                 5.2254      0.308     16.981      0.000       4.622       5.829
Over 60            -2.2106      0.172    -12.852      0.000      -2.548      -1.873
Under 40            2.3554      0.178     13.202      0.000       2.006       2.705
College            -2.8216      0.157    -17.945      0.000      -3.130      -2.513
Over 100k          -1.1143      0.179     -6.212      0.000      -1.466      -0.763
Under 50k           2.0439      0.176     11.600      0.000       1.699       2.389
High_num           -6.1761      0.173    -35.778      0.000      -6.514      -5.838
==============================================================================
Omnibus:                    49271.272   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           180966.104
Skew:                           1.950   Prob(JB):                         0.00
Kurtosis:                       7.312   Cond. No.                         27.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""